**VI. Эксплуатация**

Лекция 8. Деплоймент/развертка моделей. Введение в Deployment. Batch и online предсказание. Паттерны развертывания

Лекция 9. Практика

Лекция 10. Практика

Лекция 11. Оптимизация Inference

Чип Хьюэн Глава 10. Инфракструктура

Лекция 12. Практика

Лекция 13. Практика

Лекция 14. Практика ML в ecom

**Бонус: человеческий фактор в МЛ**

Лекция 15. Этика ИИ.

# Лекция 8. Деплоймент/развертка моделей. 

## Введение в Deployment

Deployment - процесс внедрения обученной ML-модели в production среду, где она может принимать реальные данные и делать предсказания для бизнес-приложения. Непрерывный процесс, включающий мониторинг, обновление, масштабирование.

Требования production:

    низкая latency (<100 мс)
    
    высокая throughput (тысячи запросов/сек)
    
    Reliability (99.9%+uptime)
    
    Scalability (автомасштабирование под нагрузку)
    
    Cost efficiency (оптимизация вычислительных ресурсов)
    
Проблемы интеграции 

    различные форматы моделей (PyTorch, TensorFlow)
    
    разные языки (python&Go)
    
    версионирование моделей
    
    A/B тестирование
    
    Мониторинг деградации качества
    
Жизненный цикл модели в production

    Обучение модели
    
    Офлайн оценка
    
    Подготовка (сериализация, оптимизация)
    
    Работа в тестовом режиме на ограниченном числе пользователей (Canary deploy/Shadow mode)
    
    A/B тестирование
    
    Полное развертывание
    
    Мониторинг
    
    Переобучение

## Batch vs Online Prediction

**Batch prediction** - для большого набора данных предсказания генерируются заранее.

Характеристики:

    Периодическое выполнение
    
    Сохраняются в БД/кеше
    
    Сглаживает нагрузку и оптимизирует использование ресурсов
    
    НО! предсказания устаревают и их где-то надо хранить
    
**Online prediction**

Характеристики

    Синхронный вызов API
    
    Низкая latency
    
    Обрабатывает небольшие батчи за раз
    
    Свежие предсказания на актуальных данных
    
**Streaming prediction** - непрерывная обработка потока данных (система готова к поступлению новых данных)

    Асинхронная обработка
    
    Модель обрабатывает события по мере поступления
    
    Результаты отправляются в другие системы
    
    Высокая пропускная способность
    
    Развязываем сервиси-производители и сервисы-потребители

**Hybrid подход**

## Паттерны развертывания моделей

Развертывание в облаке или на устройстве?

|Облако|Устройство|
|-|-|
|Простота развертывания и управления с помощью облачных служб|Развертывание модели
на устройстве может быть сложным|
|Затраты на облачные ресурсы могут быть высокими|Вычисления выполняются на устройстве, нет затрат на аренду облачных ресурсов|
|Есть сетевая задержка|Нет сетевой задержки|
|Обычно ответ выдается быстрее благодаря использованию более мощного оборудования|Модели МО работают медленнее|
|Меньше ограничений|Больше ограничений по памяти, потреблению аккумулятора и т. д.|
|Конфиденциальность ослабляется, потому что пользовательские данные переносятся в облако|Более высокий уровень конфиденциальности, потому что данные не покидают устройство|
|Чтобы отправлять и получать данные из облачных служб, необходимо подключение к интернету|Подключение к интернету не требуется|

Также есть фреймворки для разворачивания ml-модели в браузере


1. Модель как сервис

    Client + API + Model Service + Model
    
    fastapi
    
    зависит от сети
    
2. Встроенная модель

    Интегрирована с приложением
    
    не зависит от сети, может работать оффлайн
    
3. Модель как звисимость

    Модель загружается как библиотека/пакет, скажем, в корпоративном хранилище
    
4. Предрасчитанные данные
    
    Все возможные предсказания просчитали заранее
    
    Холодные старты, малое число товаров, пиковая нагрузка
    
5. Федеративное обучение.

    Модель обучается на устройствах пользователей без отправки данных на сервер
    
    Секретность! Т9

## Непрерывное обучение

Если дообучать модель на каждой новой порции данных

    Стоимость
    
    Катастрофическая забывчивость
    
    Восприимчивость к аномалиям
    
Новую модель надо протестировать! Желательно, сохранив предыдущее состояние.

Непрерывное обучение необходимо, когда идут редкие события, холодный старт (обучение без начальных данных). Частота обновления модели - тонкий вопрос.

Проблемы непрерывного обучения

    Доступ к новым данным
    
    Оценки
    
    Алгоритмы

## Тестирование при эксплуатации

Единственный способ убедиться, что модель показывает хорошие результаты при эксплуатации, — протестировать ее на реальном трафике.

    Теневое развертывание 
    
        В этом варианте новая модель развертывается параллельно с существующей. Каждый входной запрос передается обеим моделям, но пользователь получает предсказания только от существующей модели.
        
    A/B-тестирование

        В таком формате тестирования новая модель развертывается параллельно с существующей. Часть трафика передается новой модели, а остальные запросы — существующей
        
    Канареечный релиз
    
        Постепенное непрерывное рпспространение новой модели
        
    Тестирование чередованием
    
        Предложим пользователю на выбор оба варианта (Ничего не напоминает? RLHF и арены LLM)
        
    (Многорукие) бандиты
    
        надо было ботать тервер и случайные процессы...
        
        
        

# Лекция 11. Оптимизация Inference

## Оптимизация Inferece

Оптимизация важна - у нас есть влияние скорости работы на бизнес/загруженность ресурсов. Лишние вычисления=потери. 

Цели:

    снижение latency
    
    увеличение throughput (количество предсказаний/сек)
    
    уменьшение занятой памяти
    
    снижение издержек (износ CPU/GPU)
    
Методы оптимизации:

    Batching (группировка в батчи)
    
        эффективнее использует ресурсы НО оптимальный батч надо подбирать по обстоятельствам
        
        увеличивает throughput НО увеличивает latency
        
    Старое доброе Model Cashing - если много повторяющихся запросов
    
    Оптимизация на железе - Специализированное оборудование (TPU для TensorFlow например)
        
    Inference optimization libraries
    
    Параллелизация
    
    Оптимизация кода (использование встроенных оптимизированных функций, оптимизация циклов,   )
    
    Оптимизация ML-моделей (специфические для конкретной архитектуры эвристики)
    
    Сжатие моделей    
        

## Сжатие моделей

Большие модели не влезают в память и медленно работают.

    Квантование (уменьшаем точность висов/тренируем сразу с квантованием)
    
    Pruning - удаление 'лишних' весов
    
    Дистилляция - обучение маленькой модели имитировать большую модель
    
    LoRA Factorization
    
    Экзотика: Neural Architecture Search - AutoML technique that automates the design of artificial neural networks, reducing human intervention to create optimized, efficient models for specific tasks.
    
Best practice:

    комбинирование нескольких техник

## Очереди сообщений

Message Broker - архитектурный паттерн в распределенных системах. Приложение, которое преобразует сообщение по одному протоколу от приложения-источника к протоколу приложения-приемника.

Типы взаимодействия приложений:

    point-to-point одно приложение обращается к другому
    
    push/sub одно приложение оставляет сообщение, другое его забирает

Как организовать общение между разными приложениями? Что делать, если одно приложение упало? Нужен единый брокер

    Интеграция систем с разными протоколами
    
    Маршрутизация сообщений
    
    Надежное хранение
    
    Гарантия доставки
    
    Масштабирование
    
    Преобразование сообщений
    
    Интеграция с внешними источниками
    
        

# Чип Хьюэн Глава 10. Инфракструктура

Хранение и вычисление, ML-платформа

    В облаке, или на носителе, своими GPU?

Управление ресурсами

    планировщик, оркестратор

Управление рабочим процессом

    airflow, kuberflow

Среда разработки

    нужно версионирование (Git, MlFlow), нужна стандартизация хотя бы в рамках команды,

Развертывание

    Docker, Kubernetes, специализированные сервисы
    
Архив 

    Хранилище моделей, хранилище признаков

# Лекция 15. Этика ИИ.

Кто ставит подпись? Кто должен нести ответственность за решение, принятое ИИ?

Согласованность. Результаты работы модели не должны (сильно) противоречить друг другу. Если модель будет давать каждый раз новые фильтры, как найти понравившийся?

Что нужно проверять при взаимодействии с ИИ

    проверка на bias
    
    проверка на неучтенные группы
    
    независимая аналитика
    
    прозрачность работы модели
    
    шифрование и анонимизация
    
    план отката
    
    план мониторинга

О роли команды.

    От универсальной команды, до набора узкоспециализированных команд
    
    Кто главный?

Ответственный ИИ: примеры проблем

    Дискриминация по расе полу (модель впитала не совсем политкорректный bias)
    
    Постановка неправильной цели
    
    Отсутствие прозрачности
    
    Утечки конфиденциальных данных

Best Practice:

    Предполагать, откуда могут прийти искажения. Обучающие данные? Неправильная цель?
    
    Понимание ограничений, искусство компромиссов
    
    Продумывать заранее
    
    Версионирование. (данные, метрики, цели, этические соображения, предупреждения и рекомендации)
    
    Следить за тенденциями в ответственном ИИ